In [2]:
import torch
import torch.nn as nn
from torchvision import transforms
import sys
sys.path.append('/opt/cocoapi/PythonAPI')
from pycocotools.coco import COCO
from data_loader import get_loader
from model import EncoderCNN, DecoderRNN
import math


## TODO #1: Select appropriate values for the Python variables below.
batch_size = 128          # batch size
vocab_threshold = 5        # minimum word count threshold
vocab_from_file = False   # if True, load existing vocab file
embed_size = 512           # dimensionality of image and word embeddings
hidden_size = 512        
# number of features in hidden state of the RNN decoder
num_epochs = 3             # number of training epochs
save_every = 1             # determines frequency of saving model weights
print_every = 100          # determines window for printing average loss
log_file = 'training_log.txt'       # name of file with saved training loss and perplexity

# (Optional) TODO #2: Amend the image transform below.
transform_train = transforms.Compose([ 
    transforms.Resize(256),                          # smaller edge of image resized to 256
    transforms.RandomCrop(224),                      # get 224x224 crop from random location
    transforms.RandomHorizontalFlip(),               # horizontally flip image with probability=0.5
    transforms.ToTensor(),                           # convert the PIL Image to a tensor
    transforms.Normalize((0.485, 0.456, 0.406),      # normalize image for pre-trained model
                         (0.229, 0.224, 0.225))])

# Build data loader.
data_loader = get_loader(transform=transform_train,
                         mode='train',
                         batch_size=batch_size,
                         vocab_threshold=vocab_threshold,
                         vocab_from_file=vocab_from_file)

# The size of the vocabulary.
vocab_size = len(data_loader.dataset.vocab)
# Initialize the encoder and decoder. 
encoder = EncoderCNN(embed_size)
decoder = DecoderRNN(embed_size, hidden_size, vocab_size)

# Move models to GPU if CUDA is available. 
device = torch.device("cuda")
encoder.to(device)
decoder.to(device)

# Define the loss function. 
criterion = nn.CrossEntropyLoss().cuda() #if torch.cuda.is_available() else nn.CrossEntropyLoss()

# TODO #3: Specify the learnable parameters of the model.
params = list(decoder.parameters()) + list(encoder.embed.parameters())

# TODO #4: Define the optimizer.
optimizer = torch.optim.Adam(params, lr=0.001)

# Set the total number of training steps per epoch.
total_step = math.ceil(len(data_loader.dataset.caption_lengths) / data_loader.batch_sampler.batch_size)

loading annotations into memory...
Done (t=0.63s)
creating index...
index created!
[0/591753] Tokenizing captions...
[100000/591753] Tokenizing captions...
[200000/591753] Tokenizing captions...
[300000/591753] Tokenizing captions...
[400000/591753] Tokenizing captions...
[500000/591753] Tokenizing captions...
loading annotations into memory...
Done (t=0.63s)
creating index...
index created!
Obtaining caption lengths...


100%|███████████████████████████████████████████████████████████████████████| 591753/591753 [00:40<00:00, 14475.97it/s]


In [3]:
import torch.utils.data as data
import numpy as np
import os
import requests
import time

# Open the training log file.
f = open(log_file, 'w')

#old_time = time.time()
#response = requests.request("GET", 
#                           "http://metadata.google.internal/computeMetadata/v1/instance/attributes/keep_alive_token", 
#                           headers={"Metadata-Flavor":"Google"})

for epoch in range(1, num_epochs+1):
    
    for i_step in range(1, total_step+1):
        
        #if time.time() - old_time > 60:
        #   old_time = time.time()
        #    requests.request("POST", 
        #                    "https://nebula.udacity.com/api/v1/remote/keep-alive", 
        #                    headers={'Authorization': "STAR " + response.text})
        
        # Randomly sample a caption length, and sample indices with that length.
        indices = data_loader.dataset.get_train_indices()
        # Create and assign a batch sampler to retrieve a batch with the sampled indices.
        new_sampler = data.sampler.SubsetRandomSampler(indices=indices)
        data_loader.batch_sampler.sampler = new_sampler
        
        # Obtain the batch.
        images, captions = next(iter(data_loader))

        # Move batch of images and captions to GPU if CUDA is available.
        images = images.to(device)
        captions = captions.to(device)
        
        # Zero the gradients.
        decoder.zero_grad()
        encoder.zero_grad()
        
        # Pass the inputs through the CNN-RNN model.
        features = encoder(images)
        outputs = decoder(features, captions)
        
        # Calculate the batch loss.
        loss = criterion(outputs.view(-1, vocab_size), captions.view(-1))
        
        # Backward pass.
        loss.backward()
        
        # Update the parameters in the optimizer.
        optimizer.step()
            
        # Get training statistics.
        stats = 'Epoch [%d/%d], Step [%d/%d], Loss: %.4f, Perplexity: %5.4f' % (epoch, num_epochs, i_step, total_step, loss.item(), np.exp(loss.item()))
        
        # Print training statistics (on same line).
        print('\r' + stats, end="")
        sys.stdout.flush()
        
        # Print training statistics to file.
        f.write(stats + '\n')
        f.flush()
        
        # Print training statistics (on different line).
        if i_step % print_every == 0:
            print('\r' + stats)
            
    # Save the weights.
    if epoch % save_every == 0:
        torch.save(decoder.state_dict(), os.path.join('./models', 'decoder-%d.pkl' % epoch))
        torch.save(encoder.state_dict(), os.path.join('./models', 'encoder-%d.pkl' % epoch))

# Close the training log file.
f.close()

Epoch [1/3], Step [100/4624], Loss: 4.3252, Perplexity: 75.5804869
Epoch [1/3], Step [200/4624], Loss: 3.8964, Perplexity: 49.22278
Epoch [1/3], Step [300/4624], Loss: 4.1968, Perplexity: 66.47598
Epoch [1/3], Step [400/4624], Loss: 3.7898, Perplexity: 44.2477
Epoch [1/3], Step [500/4624], Loss: 3.3964, Perplexity: 29.8559
Epoch [1/3], Step [600/4624], Loss: 3.9209, Perplexity: 50.4442
Epoch [1/3], Step [700/4624], Loss: 2.9790, Perplexity: 19.6681
Epoch [1/3], Step [800/4624], Loss: 3.1695, Perplexity: 23.7958
Epoch [1/3], Step [900/4624], Loss: 3.6946, Perplexity: 40.2302
Epoch [1/3], Step [1000/4624], Loss: 3.0262, Perplexity: 20.6177
Epoch [1/3], Step [1100/4624], Loss: 2.8274, Perplexity: 16.9022
Epoch [1/3], Step [1200/4624], Loss: 2.9885, Perplexity: 19.85627
Epoch [1/3], Step [1300/4624], Loss: 2.9350, Perplexity: 18.8213
Epoch [1/3], Step [1400/4624], Loss: 3.5940, Perplexity: 36.3810
Epoch [1/3], Step [1500/4624], Loss: 2.7146, Perplexity: 15.0982
Epoch [1/3], Step [1600/4624

Epoch [3/3], Step [3500/4624], Loss: 2.1766, Perplexity: 8.81660
Epoch [3/3], Step [3600/4624], Loss: 2.3881, Perplexity: 10.8923
Epoch [3/3], Step [3700/4624], Loss: 2.0044, Perplexity: 7.42207
Epoch [3/3], Step [3800/4624], Loss: 2.0154, Perplexity: 7.50408
Epoch [3/3], Step [3900/4624], Loss: 2.1204, Perplexity: 8.33417
Epoch [3/3], Step [4000/4624], Loss: 2.2195, Perplexity: 9.20293
Epoch [3/3], Step [4100/4624], Loss: 2.3253, Perplexity: 10.2292
Epoch [3/3], Step [4200/4624], Loss: 2.1610, Perplexity: 8.68013
Epoch [3/3], Step [4300/4624], Loss: 2.4098, Perplexity: 11.1314
Epoch [3/3], Step [4400/4624], Loss: 2.0600, Perplexity: 7.84587
Epoch [3/3], Step [4500/4624], Loss: 2.1039, Perplexity: 8.19845
Epoch [3/3], Step [4600/4624], Loss: 2.3446, Perplexity: 10.4288
Epoch [3/3], Step [4624/4624], Loss: 2.0171, Perplexity: 7.51683